In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
856,Ann-Margret did the best job she has ever done...,positive
55,Lovingly crafted and terribly interesting to w...,positive
671,"A ""40 foot long"" giant mutant squid with five ...",negative
893,"Much like the comedy duo of its title, ""The Su...",positive
546,I watched this film in shire joy.<br /><br />T...,positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
856,ann margret best job ever done history film ma...,positive
55,lovingly crafted terribly interesting watch ga...,positive
671,foot long giant mutant squid five tentacle raz...,negative
893,much like comedy duo title the sunshine boy be...,positive
546,watched film shire joy br br this possibly one...,positive


In [6]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
856,ann margret best job ever done history film ma...,1
55,lovingly crafted terribly interesting watch ga...,1
671,foot long giant mutant squid five tentacle raz...,0
893,much like comedy duo title the sunshine boy be...,1
546,watched film shire joy br br this possibly one...,1


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/shahil04/insurance_project_mlops.mlflow')
dagshub.init(repo_owner='shahil04', repo_name='insurance_project_mlops', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")



❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\hp\.conda\envs\mlopss\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=386da9df-8d84-4cd5-bd86-062e6e99584f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a45094a123ba8082e67b1404ab0a1fbd9f8a75d381c944223a3a612d7a09039b




Accessing as shahil04

Initialized MLflow to track repo "shahil04/insurance_project_mlops"

Repository shahil04/insurance_project_mlops initialized!

2025/05/16 20:21:59 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7397ef34b84f4181a3106d175ea6f5c9', creation_time=1747407122683, experiment_id='0', last_update_time=1747407122683, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [13]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-05-16 20:22:26,517 - INFO - Starting MLflow run...


2025-05-16 20:22:27,374 - INFO - Logging preprocessing parameters...
2025-05-16 20:22:28,768 - INFO - Initializing Logistic Regression model...
2025-05-16 20:22:28,768 - INFO - Fitting the model...
2025-05-16 20:22:28,836 - INFO - Model training complete.
2025-05-16 20:22:28,847 - INFO - Logging model parameters...
2025-05-16 20:22:29,259 - INFO - Making predictions...
2025-05-16 20:22:29,265 - INFO - Calculating evaluation metrics...
2025-05-16 20:22:29,328 - INFO - Logging evaluation metrics...
2025-05-16 20:22:31,109 - INFO - Saving and logging the model...
2025/05/16 20:22:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-16 20:22:48,296 - INFO - Model training and logging completed in 20.94 seconds.
2025-05-16 20:22:48,314 - INFO - Accuracy: 0.624
2025-05-16 20:22:48,316 - INFO - Precision: 0.5882352941176471
2025-05-16 20:22:48,320 - INFO - Reca

🏃 View run puzzled-bat-270 at: https://dagshub.com/shahil04/insurance_project_mlops.mlflow/#/experiments/0/runs/1964221cefcf4badb139138ce5f3c761
🧪 View experiment at: https://dagshub.com/shahil04/insurance_project_mlops.mlflow/#/experiments/0
